In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [2]:
class NN:
    def __init__(self, layers, nodes_per_layer, data):
        super().__init__()
        self.layers = layers
        self.nodes_per_layer = nodes_per_layer
        self.m, self.n = data.shape
        np.random.shuffle(data)

        test_data = data[0:1000].T
        Y_test = test_data[0]
        X_test = test_data[1:self.n]

        train_data = data[1000:self.m].T
        self.Y_train = train_data[0]
        X_train = train_data[1:self.n]
        self.X_train = X_train / 255
        self.output_size = self.Y_train.max() + 1
    
    def init_params(self):
        weights = []
        biases = []

        if self.layers == 1:
            weights.append(np.random.rand(self.output_size, self.n - 1) - 0.5)
            biases.append(np.random.rand(self.output_size, 1))
        else:
            for i in range(self.layers):
                if i == 0:
                    weights.append(np.random.rand(self.nodes_per_layer, self.n - 1) - 0.5)
                    biases.append(np.random.rand(self.nodes_per_layer, 1))
                elif i == self.layers - 1:
                    weights.append(np.random.rand(self.output_size, self.nodes_per_layer) - 0.5)
                    biases.append(np.random.rand(self.output_size, 1))
                else: 
                    weights.append(np.random.rand(self.nodes_per_layer, self.nodes_per_layer) - 0.5)
                    biases.append(np.random.rand(self.nodes_per_layer, 1))

        return weights, biases

    def ReLU(self, Z):
        return np.maximum(Z, 0)
    
    def ReLU_deriv(self, Z):
        return Z > 0
    
    def soft_max(self, Z):
        A = np.exp(Z) / sum(np.exp(Z))
        return A
    
    def forward_prop(self, weights, biases, X):
        Z = []
        A = []
        for i in range(self.layers):
            if(i == 0):
                Z.append(weights[i].dot(X) + biases[i])
                A.append(self.ReLU(Z[i]))
            elif(i == self.layers - 1):
                Z.append(weights[i].dot(A[i-1]) + biases[i])
                A.append(self.soft_max(Z[i]))
            else: 
                Z.append(weights[i].dot(A[i-1]) + biases[i])
                A.append(self.ReLU(Z[i]))

        return Z, A
    
    def one_hot(self, Y):
        one_hot_Y = np.zeros((Y.size, Y.max() + 1))
        one_hot_Y[np.arange(Y.size), Y] = 1
        one_hot_Y = one_hot_Y.T
        return one_hot_Y
    
    def back_prop(self, Z, A, weights, X, Y):
        one_hot_Y = self.one_hot(Y)
        dZ = []
        dW = []
        db = []
        
        if self.layers == 1:
            dZ.append(A[0] - one_hot_Y)
            dW.append(1 / self.m * dZ[0].dot(X.T))
            db.append(1 / self.m * np.sum(dZ[0]))

        else:
            for i in reversed(range(self.layers)):
                current_dz = self.layers - (i + 1)
                next_dz = self.layers - (i + 2)
    #           First round
                if(i == self.layers - 1):
                    dZ.append(A[i] - one_hot_Y)
                    dW.append(1 / self.m * dZ[current_dz].dot(A[i-1].T))
    #           Last round
                elif(i == 0):
                    dZ.append(weights[i+1].T.dot(dZ[next_dz]) * self.ReLU_deriv(Z[i]))
                    dW.append(1 / self.m * dZ[current_dz].dot(X.T))
    #           Everything in between
                else:
                    dZ.append(weights[i+1].T.dot(dZ[next_dz]) * self.ReLU_deriv(Z[i]))
                    dW.append(1 / self.m * dZ[current_dz].dot(A[i-1].T))
                db.append(1 / self.m * np.sum(dZ[current_dz]))
        dW.reverse()
        db.reverse()

        return dW, db
    
    def update_params(self, weights, biases, dW, db, alpha):
        for i in range(self.layers):
            weights[i] -= (alpha  * dW[i])
            biases[i] -= (alpha * db[i])
        return weights, biases
    
    def get_predictions(self, A):
        return np.argmax(A, 0)

    def get_accuracy(self, predictions, Y):
        return np.sum(predictions == Y) / Y.size

    def gradient_descent(self, iterations, alpha):
        X = self.X_train
        Y = self.Y_train
        
        weights, biases = self.init_params()
        for i in range(iterations):
            Z, A = self.forward_prop(weights, biases, X)
            dW, db = self.back_prop(Z, A, weights, X, Y)
            weights, biases = self.update_params(weights, biases, dW, db, alpha)
            if(i % 50 == 0):
                print(f"Iteration: {i}")            
                print(f"Accuracy: {self.get_accuracy(self.get_predictions(A[self.layers-1]), Y)}")
                
        self.weights = weights
        self.biases = biases
    
    def make_predictions(self, X):
        Z, A = self.forward_prop(nn.weights, nn.biases, X)
        predictions = self.get_predictions(A[self.layers-1])
        return predictions
    
    def test_prediction(self, index):
        current_image = self.X_train[:, index, None]
        prediction = self.make_predictions(current_image, nn.weights, nn.biases)
        label = self.Y_train[index]
        print("Prediction: ", prediction)
        print("Label: ", label)

        current_image = current_image.reshape((28, 28)) * 255
        plt.gray()
        plt.imshow(current_image, interpolation='nearest')
        plt.show()

In [3]:
data = pd.read_csv('data/train.csv')
data = np.array(data)

In [ ]:
nn = NN(2, 10, data)
nn.gradient_descent(100, 0.1)
nn.test_prediction(12)

Iteration: 0
Accuracy: 0.10248780487804877
